In [3]:
#packages for Model
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D,AveragePooling2D, MaxPooling2D, ZeroPadding2D
from keras.layers.core import Activation,Flatten,Dropout,Dense
from keras import backend as K

In [2]:
#packages for training
import matplotlib
matplotlib.use("Agg")
from keras.preprocessing.image import ImageDataGenerator #数据扩张
from keras import optimizers #优化器
#from pyimagesearch.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os
import shutil

Using TensorFlow backend.


In [4]:
model=Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(2,2))

# model.add(Conv2D(128,(3,3),activation='relu'))
# model.add(BatchNormalization(axis=-1))
# model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(128,activation='relu'))#512也试过，plot的图就是512的时候，后来改成了32训不动了太慢了，就没训完。
model.add(Dense(1,activation='sigmoid'))

In [5]:
from keras.optimizers import Adam
EPOCHS=30 #网络迭代次数
INIT_LR=1e-4 #Adam的初始学习率
optimizer=Adam(lr=INIT_LR,beta_1=0.9, beta_2=0.999, epsilon=1e-08,decay=INIT_LR/EPOCHS)
model.compile(loss='binary_crossentropy',metrics=['acc'],optimizer=optimizer)

In [6]:
train_dir='newdataset/train2'
val_dir='newdataset/validation2'
train_datagen=ImageDataGenerator(rescale=1./255,rotation_range=10,width_shift_range=0.1,
                       height_shift_range=0.1,shear_range=0.1,zoom_range=0.1,
                       horizontal_flip=True,fill_mode="nearest")#由于是街景，不能变动太多
test_datagen=ImageDataGenerator(rescale=1./255)
train_generator=train_datagen.flow_from_directory(train_dir,target_size=(150,150),batch_size=64,class_mode='binary')
validation_generator=test_datagen.flow_from_directory(val_dir,target_size=(150,150),batch_size=64,class_mode='binary')

Found 2247 images belonging to 2 classes.
Found 615 images belonging to 2 classes.


In [8]:
from PIL import Image
history=model.fit_generator(train_generator,steps_per_epoch=10,epochs=30,validation_data=validation_generator,validation_steps=20)

Epoch 1/30
10/10 [==============================] - 146s 15s/step - loss: 0.8246 - acc: 0.7083 - val_loss: 1.1629 - val_acc: 0.6211
Epoch 2/30
 2/10 [=====>........................] - ETA: 1:13 - loss: 1.0375 - acc: 0.6406

C:\ProgramData\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:780: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


10/10 [==============================] - 156s 16s/step - loss: 0.8566 - acc: 0.6453 - val_loss: 0.7990 - val_acc: 0.6033
Epoch 3/30
10/10 [==============================] - 146s 15s/step - loss: 0.6029 - acc: 0.6859 - val_loss: 0.5825 - val_acc: 0.6797
Epoch 4/30
10/10 [==============================] - 140s 14s/step - loss: 0.5309 - acc: 0.7341 - val_loss: 0.7062 - val_acc: 0.6130
Epoch 5/30
10/10 [==============================] - 147s 15s/step - loss: 0.5463 - acc: 0.7594 - val_loss: 0.6748 - val_acc: 0.6407
Epoch 6/30
10/10 [==============================] - 147s 15s/step - loss: 0.5156 - acc: 0.7297 - val_loss: 0.6339 - val_acc: 0.6520
Epoch 7/30
10/10 [==============================] - 147s 15s/step - loss: 0.5281 - acc: 0.7422 - val_loss: 0.7121 - val_acc: 0.6228
Epoch 8/30
10/10 [==============================] - 163s 16s/step - loss: 0.5273 - acc: 0.7328 - val_loss: 0.5752 - val_acc: 0.6878
Epoch 9/30
10/10 [==============================] - 156s 16s/step - loss: 0.5030 - acc:

KeyboardInterrupt: 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(1,len(acc)+1)

plt.plot(epochs,acc,'bo',label='Training acc')
plt.plot(epochs,val_acc,'b',label='Validation acc')
plt.title('Training and validation accuracy')
plt.yticks([0.5,0.6,0.7,0.8,0.9,1])
plt.legend()
plt.figure()

plt.plot(epochs,loss,'bo',label='Training loss')
plt.plot(epochs,val_loss,'b',label='Validation loss')
plt.title('Training and validation loss')
plt.yticks([0.5,0.6,0.7,0.8,0.9,1])
plt.legend()
plt.figure()

plt.show()
plt.savefig('spacesyntax2.jpg')

In [ ]:
model.save('spacesyntax2.ckpt')

In [47]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='modelusing13-3.png', show_shapes=True)